Imports + Auth

In [32]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time

subscription_key = "74e4096a2d984a61a75e1c640ba6e25e"
endpoint = "https://ai-vision-andrei-diaconescu.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

Processing the image

In [33]:
# img = open("images/test1.png", "rb")
img = open("images/test2.jpeg", "rb")
read_response = computervision_client.read_in_stream(
    image=img,
    mode="Printed",
    raw=True
)
# print(read_response.as_dict())

operation_id = read_response.headers['Operation-Location'].split('/')[-1]
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

result = []
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            result.append(line.text)

Lucces in resolvarea
TEMELOR la
LABORA toarele de
Inteligenta Artificialà!


Evaluation

In [34]:
def longest_common_subsequence(str1, str2):
  dp = [[0 for _ in range(len(str2) + 1)] for _ in range(len(str1) + 1)]
  for i in range(1, len(str1) + 1):
    for j in range(1, len(str2) + 1):
      if str1[i - 1] == str2[j - 1]:
        dp[i][j] = 1 + dp[i - 1][j - 1]
      else:
        dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

  return dp[-1][-1]

def hamming_dist(str1, str2):
  cnt = 0
  for i in range(min(len(str1), len(str2))):
    if str1[i] != str2[i]:
      cnt += 1
  return cnt

def hamming_word_dist(str1, str2):
  cnt = 0
  words_str1 = str1.split(' ')
  words_str2 = str2.split(' ')
  for i in range(min(len(words_str1), len(words_str2))):
    if words_str1[i] != words_str2[i]:
      cnt += 1
  return cnt

def levenshtein_dist(s1, s2):
    matrix = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]

    for i in range(len(s1) + 1):
        matrix[i][0] = i
    for j in range(len(s2) + 1):
        matrix[0][j] = j

    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if s1[i - 1] == s2[j - 1]:
                cost = 0
            else:
                cost = 1
            matrix[i][j] = min(matrix[i-1][j] + 1,    
                               matrix[i][j-1] + 1,
                               matrix[i-1][j-1] + cost)  

    return matrix[len(s1)][len(s2)]  

def wer_dist(ref, hyp):
    ref_words = ref.split()
    hyp_words = hyp.split()
    
    N = len(ref_words)
    M = len(hyp_words)
    dp = [[0] * (M + 1) for _ in range(N + 1)]
    
    for i in range(N + 1):
        dp[i][0] = i
    for j in range(M + 1):
        dp[0][j] = j
    
    for i in range(1, N + 1):
        for j in range(1, M + 1):
            if ref_words[i - 1] == hyp_words[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j - 1], dp[i - 1][j], dp[i][j - 1]) + 1
    
    wer_value = dp[N][M] / len(ref_words) * 100
    return wer_value 



# ground_truth = ["Google Cloud", "Platform"]
ground_truth = ["Succes in rezolvarea", "tEMELOR la", "LABORAtoaree de", "Inteligenta Artificiala!"]

lcs = longest_common_subsequence("".join(result), "".join(ground_truth))
hd = hamming_dist("".join(result), "".join(ground_truth))
ld = levenshtein_dist("".join(result), "".join(ground_truth))
wer = wer_dist(" ".join(result), " ".join(ground_truth))
hd_word = hamming_word_dist(" ".join(result), " ".join(ground_truth))

print("Evaluations")
print("LCS:", lcs)
print("Hamming:", hd)
print("Levenshtein:", ld)
print("Word Error Rate:", wer)
print("Hamming Word:", hd_word)

Evaluations
LCS: 65
Hamming: 32
Levenshtein: 6
Word Error Rate: 60.0
Hamming Word: 7


Localisation Comparison

In [35]:
def aria_patrulaterului(puncte):

    x1 = puncte[0]
    y1 = puncte[1]
    x2 = puncte[2]
    y2 = puncte[3]
    x3 = puncte[4]
    y3 = puncte[5]
    x4 = puncte[6]
    y4 = puncte[7]
    return abs((x1*y2 + x2*y3 + x3*y4 + x4*y1 - x2*y1 - x3*y2 - x4*y3 - x1*y4) / 2)

def aria_intersectiei(patrulater1, patrulater2):
    puncte_intersectie = [
        max(patrulater1[0], patrulater2[0]),
        max(patrulater1[1], patrulater2[1]),
        
        min(patrulater1[2], patrulater2[2]),
        max(patrulater1[3], patrulater2[3]),
        
        min(patrulater1[4], patrulater2[4]),
        min(patrulater1[5], patrulater2[5]),
        
        max(patrulater1[6], patrulater2[6]),
        min(patrulater1[7], patrulater2[7])
        
    ]
    aria_intersectiei = aria_patrulaterului(puncte_intersectie)
    return aria_intersectiei

def IoU(patrulater1, patrulater2):
    aria1 = aria_patrulaterului(patrulater1)
    aria2 = aria_patrulaterului(patrulater2)
    intersectie = aria_intersectiei(patrulater1, patrulater2)
    uniune = aria1 + aria2 - intersectie
    
    if uniune == 0:
        return 0
    
    return intersectie / uniune


# bounding_boxes_truth = [
#     [177.0, 43.0, 414.0, 43.0, 414.0, 104.0, 177.0, 104.0],
#     [237.0, 113.0, 347.0, 113.0, 347.0, 150.0, 237.0, 150.0]]

bounding_boxes_truth = [
  [79.0, 299.0, 1333.0, 299.0, 1333.0, 462.0, 79.0, 462.0],
  [130.0, 580.0, 1045.0, 580.0, 1045.0, 722.0, 130.0, 722.0],
  [82.0, 920.0, 998.0, 920.0, 998.0, 1026.0, 82.0, 1026.0],
  [105.0, 1128.0, 1452.0, 1128.0, 1452.0, 1368.0, 105.0, 1368.0],
  ]

bound_boxes_detected = []
for text_result in read_result.analyze_result.read_results:
    for line in text_result.lines:
        bound_boxes_detected.append(line.bounding_box)

for i in range(len(bounding_boxes_truth)):
    print(IoU(bounding_boxes_truth[i], bound_boxes_detected[i]))

0.8190826249406705
0.910353012093584
0.8346515866007853
0.5646560257362039


Enhancing the text recognition for test2.jpeg

In [36]:
from PIL import Image, ImageFilter, ImageEnhance

img = Image.open('images/test2.jpeg')

img = img.convert('L')

enhancer = ImageEnhance.Contrast(img)
img = enhancer.enhance(2.0)

threshold = 140
img = img.point(lambda p: p > threshold and 255)

img.save('images/test2_enhanced.jpeg')

Enhancing the text recognition for test1.png

In [37]:
from PIL import Image

threshold = 170 

def is_black_or_gray(r, g, b):
    avg_intensity = (r + g + b) / 3
    if avg_intensity < threshold:
        return True
    else:
        return False
    
image = Image.open('images/test1.png')

image = image.convert('RGB')

binarized_image = Image.new('1', image.size, 255) 

for x in range(image.width):
    for y in range(image.height):
        r, g, b = image.getpixel((x, y))
        if is_black_or_gray(r, g, b):
            binarized_image.putpixel((x, y), 0) 

# Save the binarized image
binarized_image.save('images/test1_enhanced.png')

In [38]:
# img = open("images/test1_enhanced.png", "rb")
img = open("images/test2_enhanced.jpeg", "rb")

read_response = computervision_client.read_in_stream(
    image=img,
    mode="Printed",
    raw=True
)

operation_id = read_response.headers['Operation-Location'].split('/')[-1]
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

result = []
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            result.append(line.text)

succes in resolvarea
TEMELOR la
LABORA toarele de
Inteligenta Artificialà!
